# Machine Learning Processing

### Contents

 - I. [Data Importing & Processing](#Data-Import-and-Processing)
 - II. [Modeling Plan](#Modeling-Plan)
     - [Iteration 0 - Unprocessed](#Iteration-0:-Non-Scaled-Data-for-Tree-based-Boosting-Models)
     - [Iteration 1 - Scaled](#Iteration-1:-Baby's-First-Pipe)
     - [Iteration 2 - PCA](#Iteration-2:-Give-the-Computer-a-Chance)
     - [Iteration 3 - PCA Boosted](#Iteration-3:-Using-PCA-to-Boost-Critical-Elements-of-DataFrame)
         - [Progress So Far](#Progress-Report-Card)
     - [Iteration 1 Optimization](#Iteration-1-Optimizations)
     - [Iteration 2 Optimization](#Iteration-2-Optimizations)
         - [Second Report Card](#Post-Optimization-Report-Card)
 - III. [PCA Weighted Approach](#PCA-Boosting-Technique)
     - [Non-Optimized](#PCABoosted-Modeling)
     - [GridSearch Optimization](#Optimized-PCABoosted-Modeling)
     - [General Report Card](#PCABoosted-Report-Card)
 - IV. [Results & Discussion](#Results-&-Discussion)
     - [Permutation Importance]
     - [Predictions]
     - [Business Relevants & Benefits]

In [1]:
# Import packages etc.
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pickle

## Data Import and Processing

In [2]:
Xdata = pickle.load(open('Xtrain.pickle', 'rb'))
ydata = pickle.load(open('ytrain.pickle', 'rb'))

In [3]:
from sklearn.model_selection import train_test_split

# Split the data into train & test sets for supervised learning
Xtrain, Xtest, ytrain, ytest = train_test_split(Xdata, ydata,
                                               test_size=0.2,
                                               random_state=430)

# Quick Heads Up
print('Training Data shape: ' + str(Xtrain.shape))
print('Test Data Shape: ' + str(Xtest.shape))

Training Data shape: (4328, 71)
Test Data Shape: (1082, 71)


## Modeling Plan
The general strategy of our approach incorporates an initial, broad spectrum testing of models, including the following.
 - Logistic Regression
 - KNN
 - Discriminant Analysis
 - Naive Bayes
 - SVM
 - CatBoost
 - Light GBM

This grouping should represent a decent collection of well-worn, classical methods with exceptionally long history & documentation as well as newer, useful methodologies. Of particular interest to this last point will be **Catboost & Light GBM**, two Gradient Boosting models that have shown a great deal of promise in terms of efficiency & accuracy.

### Iterative Strategy | What We Care About 
 - Iteration 0: Non-Scaled models - Naive modeling with tree-based boosting models
 - Iteration 1: Scaled models
 - Iteration 2: PCA transformed models
 - Iteration 3: PCA Boosting models (PCA features reattached to original data)
 
### Naming Procedure
We can abbreviate the model name alongside iteration number to keep track (eg. "cb0" for "Catboost, Iteration 0)

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

### Iteration 0: Non-Scaled Data for Tree-based Boosting Models
[Back to Top](#Machine-Learning-Processing)

Both Catboost & Light GBM are capable of handling unscaled and even non-numeric data, and encourage testing their models with datasets that are not extensively transformed. As such, for these 2 models, we have run a separate iteration with non-scaled data. For further reference, please consult their respective documentation. Here we will process these for some initial yield results.

In [46]:
# IT0: CatBoost Classifier
cb_0 = CatBoostClassifier(verbose = False)
cb_0.fit(Xtrain, ytrain)
cb_0_train, cb_0_test = cb_0.score(Xtrain, ytrain), cb_0.score(Xtest, ytest)

cb_grid = {'depth': [5, 6, 7, 8, 9],
           'min_data_in_leaf': [1, 2, 4]}
cb_0_opt = GridSearchCV(cb_0, cb_grid, n_jobs=-2, cv=3)
cb_0_opt.fit(Xtrain, ytrain)
cb_0_opt_train, cb_0_opt_test, cb_0_opt_params = cb_0_opt.score(Xtrain, ytrain), cb_0_opt.score(Xtest, ytest), cb_0_opt.best_params_

# print('CatBoost Train R2 Score:', cb_0_train)
# print('CatBoost Test R2 Score:', cb_0_test)
# print('CB Optimized Train Score:', cb_0_opt.score(Xtrain, ytrain))
# print('CB Optimized Test Score:', cb_0_opt.score(Xtest, ytest))
print('Best Params:', cb_0_opt.best_params_)

Best Params: {'depth': 8, 'min_data_in_leaf': 1}


In [37]:
# IT0: Light Gradient Boosting Machine Classifier
lgbm_0 = LGBMClassifier()
lgbm_0.fit(Xtrain, ytrain.values.ravel())
lgbm_0_train, lgbm_0_test = lgbm_0.score(Xtrain, ytrain.values.ravel()), lgbm_0.score(Xtest, ytest.values.ravel())

lgbm_grid = {'num_leaves': [44, 45, 46, 47, 48],
             'max_depth': [9, 10, 11, 12, 13],
             'min_data_in_leaf': [15, 16, 17, 18, 19]}
lgbm_0_opt = GridSearchCV(lgbm_0, lgbm_grid, n_jobs   = -2, cv=3)
lgbm_0_opt.fit(Xtrain, ytrain.values.ravel())
lgbm_0_opt_train, lgbm_0_opt_test, lgbm_0_opt_params = lgbm_0_opt.score(Xtrain, ytrain.values.ravel()), lgbm_0_opt.score(Xtest, ytest.values.ravel()), lgbm_0_opt.best_params_

print('LightGBM Boost Train R2 Score:', lgbm_0_train)
print('LightGBM Boost Test R2 Score:', lgbm_0_test)
print('LightGBM Optimized Train Score:', lgbm_0_opt_train)
print('LightGBM Optimized Test Score:', lgbm_0_opt_test)
print('Best Params:', lgbm_0_opt.best_params_)

[LightGBM] [Warning] min_data_in_leaf is set=16, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=16
LightGBM Boost Train R2 Score: 1.0
LightGBM Boost Test R2 Score: 0.9279112754158965
LightGBM Optimized Train Score: 1.0
LightGBM Optimized Test Score: 0.933456561922366
Best Params: {'max_depth': 12, 'min_data_in_leaf': 16, 'num_leaves': 48}


In [54]:
modellist = [ele for ele in ['cb', 'lgbm', 'lgr', 'knn', 'lda', 'gnb', 'svc'] for i in range(16)]
iterlist = [ele for ele in ['_0', '_1', '_2', '_3'] for i in range(2)] * 14
optlist = [ele for ele in ['', '_opt'] for i in range(8)] * 7
traintestlist = ['_train', '_test'] * 56
for idx in range(len(modellist)):
    modellist[idx] = modellist[idx] + iterlist[idx] + optlist[idx] + traintestlist[idx]

scores = []
for idx in range(len(modellist)):
    try:
        scores.append(locals()[modellist[idx]])
    except:
        scores.append(np.nan)


def scoring_table(scoredata):
    columns = [[ele for ele in ['Iteration0', 'Iteration1', 'Iteration2', 'Iteration3'] for i in range(2)],
           [ele for ele in ['NonScaled', 'Scaled', 'PCA', 'PCABoost'] for i in range(2)],
           ['Train', 'Test'] * 4
          ]

    c_tuples = list(zip(*columns))
    multi_columns = pd.MultiIndex.from_tuples(c_tuples)

    index = [[ele for ele in ['CatBoost', 'LightGBM', 'Logistic', 'KNN', 'LDA', 'GNB', 'SVC'] for i in range (2)], 
             ['default', 'optimized'] * 7
            ]

    i_tuples = list(zip(*index))
    multi_index = pd.MultiIndex.from_tuples(i_tuples)

    scores_df = pd.DataFrame(np.array(scoredata).reshape((len(multi_index),len(multi_columns))), 
                 index=multi_index,
                 columns=multi_columns)
    return scores_df

In [ ]:
scoring_table(scores)

### Iteration 1: Baby's First Pipe
[Back to Top](#Machine-Learning-Processing)

Our goal here will be to test the results with some initial scaling built into a pipeline. 

In [ ]:
# IT1: CatBoost Classifier
cb_1 = Pipeline(steps=[('scaler', StandardScaler()),
                     ('catboost', CatBoostClassifier(random_state=430, verbose = False))])
cb_1.fit(Xtrain, ytrain)
cb_1_train, cb_1_test = cb_1.score(Xtrain, ytrain), cb_1.score(Xtest, ytest)

cb_grid = {'catboost__depth': [6, 8, 10],
           'catboost__min_data_in_leaf': [1, 2, 4]}
cb_1_opt = GridSearchCV(cb_1, cb_grid, n_jobs=-2, cv=3)
cb_1_opt.fit(Xtrain, ytrain)
cb_1_opt_train, cb_1_opt_test, cb_1_opt_params = cb_1_opt.score(Xtrain, ytrain), cb_1_opt.score(Xtest, ytest), cb_1_opt.best_params_

# print('CatBoost Train R2 Score:', cb_1_train)
# print('CatBoost Test R2 Score:', cb_1_test)
# print('CB Optimized Train Score:', cb_1_opt.score(Xtrain, ytrain))
# print('CB Optimized Test Score:', cb_1_opt.score(Xtest, ytest))
print('Best Params:', cb_1_opt.best_params_)

In [48]:
# IT1: Light Gradient Boosting Machine Classifier
lgbm_1 = Pipeline(steps=[('scaler', StandardScaler()),
                       ('lightgbm', LGBMClassifier(n_jobs=-2, random_state=430))])
lgbm_1.fit(Xtrain, ytrain.values.ravel())
lgbm_1_train, lgbm_1_test = lgbm_1.score(Xtrain, ytrain.values.ravel()), lgbm_1.score(Xtest, ytest.values.ravel())

lgbm_grid = {'lightgbm__num_leaves': [44, 46, 48],
             'lightgbm__max_depth': [9, 11, 13],
             'lightgbm__min_data_in_leaf': [15, 17, 19]}
lgbm_1_opt = GridSearchCV(lgbm_1, lgbm_grid, n_jobs=-2, cv=3)
lgbm_1_opt.fit(Xtrain, ytrain.values.ravel())
lgbm_1_opt_train, lgbm_1_opt_test, lgbm_1_opt_params = lgbm_1_opt.score(Xtrain, ytrain.values.ravel()), lgbm_1_opt.score(Xtest, ytest.values.ravel()), lgbm_1_opt.best_params_

print('LightGBM Boost Train R2 Score:', lgbm_1_train)
print('LightGBM Boost Test R2 Score:', lgbm_1_test)
print('LightGBM Optimized Train Score:', lgbm_1_opt_train)
print('LightGBM Optimized Test Score:', lgbm_1_opt_test)
print('Best Params:', lgbm_1_opt.best_params_)

KeyboardInterrupt: 

In [42]:
# IT1: Logistic Regression
lgr_1 = Pipeline(steps=[('scaler', StandardScaler()),
                        ('logistic', LogisticRegression(n_jobs=-2))])
lgr_1.fit(Xtrain, ytrain.values.ravel())
lgr_1_train, lgr_1_test = lgr_1.score(Xtrain, ytrain.values.ravel()), lgr_1.score(Xtest, ytest.values.ravel())

lgr_grid = {'logistic__C': np.linspace(-4, 4, 50),
       'logistic__penalty': ['l1', 'l2']}
lgr_1_opt = GridSearchCV(lgr_1, lgr_grid, n_jobs=-2, cv=3)
lgr_1_opt.fit(Xtrain, ytrain.values.ravel())
lgr_1_opt_train, lgr_1_opt_test, lgr_1_opt_params = lgr_1_opt.score(Xtrain, ytrain.values.ravel()), lgr_1_opt.score(Xtest, ytest.values.ravel()), lgr_1_opt.best_params_
print('Best Params:', lgr_1_opt.best_params_)

C:\Users\gottl\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.93438152        nan 0.93484336
        nan 0.9343812         nan 0.93438088        nan 0.93414972
        nan 0.93368756        nan 0.93345639        nan 0.93299439
        nan 0.93299439        nan 0.93322555        nan 0.93322555
        nan 0.93322555        nan 0.93322555        n

Best Params: {'logistic__C': 0.24489795918367285, 'logistic__penalty': 'l2'}


In [43]:
# IT1: K-Nearest Neighbors
knn_1 = Pipeline(steps=[('scaler', StandardScaler()),
                      ('knn', KNeighborsClassifier(n_jobs=-2))])
knn_1.fit(Xtrain, ytrain.values.ravel())
knn_1_train, knn_1_test = knn_1.score(Xtrain, ytrain.values.ravel()), knn_1.score(Xtest, ytest.values.ravel())

ksqrt = int(np.round(np.sqrt(Xtrain.shape[0])))       # Get sqrt of observations for k
knn_grid = {'knn__n_neighbors': [ksqrt - 5, ksqrt - 3, ksqrt, ksqrt + 3, ksqrt + 5],
       'knn__weights': ['uniform', 'distance'],
       'knn__metric': ['euclidean', 'manhattan']}
knn_1_opt = GridSearchCV(knn_1, knn_grid, n_jobs=-2, cv=3)
knn_1_opt.fit(Xtrain, ytrain.values.ravel())
knn_1_opt_train, knn_1_opt_test, knn_1_opt_params = knn_1_opt.score(Xtrain, ytrain.values.ravel()), knn_1_opt.score(Xtest, ytest.values.ravel()), knn_1_opt.best_params_
print('Best Params:', knn_1_opt.best_params_)

Best Params: {'knn__metric': 'euclidean', 'knn__n_neighbors': 61, 'knn__weights': 'distance'}


In [45]:
# IT1: Linear Discriminant Analysis
lda_1 = Pipeline(steps=[('scaler', StandardScaler()),
                      ('lda', LinearDiscriminantAnalysis())])
lda_1.fit(Xtrain, ytrain.values.ravel())
lda_1_train, lda_1_test = lda_1.score(Xtrain, ytrain.values.ravel()), lda_1.score(Xtest, ytest.values.ravel())

lda_grid = {'lda__solver': ['svd', 'lsqr', 'eigen']}
lda_1_opt = GridSearchCV(lda_1, lda_grid, n_jobs=-2, cv=3)
lda_1_opt.fit(Xtrain, ytrain.values.ravel())
lda_1_opt_train, lda_1_opt_test, lda_1_opt_params = lda_1_opt.score(Xtrain, ytrain.values.ravel()), lda_1_opt.score(Xtest, ytest.values.ravel()), lda_1_opt.best_params_
print('Best Params:', lda_1_opt.best_params_)

Best Params: {'lda__solver': 'lsqr'}


In [ ]:
# IT1: Gaussian Naive Bayes
gnb_1 = Pipeline(steps=[('scaler', StandardScaler()),
                      ('gnb', GaussianNB())])
gnb_1.fit(Xtrain, ytrain.values.ravel())
gnb_1_train, gnb_1_test = gnb_1.score(Xtrain, ytrain.values.ravel()), gnb_1.score(Xtest, ytest.values.ravel())

gnb_grid = {'gnb__var_smoothing': np.logspace(0, -9, num = 100)}
gnb_1_opt = GridSearchCV(gnb_1, gnb_grid, n_jobs=-2, cv=3)
gnb_1_opt.fit(Xtrain, ytrain.values.ravel())
gnb_1_opt_train, gnb_1_opt_test, gnb_1_opt_params = gnb_1_opt.score(Xtrain, ytrain.values.ravel()), gnb_1_opt.score(Xtest, ytest.values.ravel()), gnb_1_opt.best_params_
print('Best Params:', gnb_1_opt.best_params_)

In [ ]:
# IT1: Support Vector Machine Classifier
svc_1 = Pipeline(steps=[('scaler', StandardScaler()),
                      ('svc', SVC())])
svc_1.fit(Xtrain, ytrain.values.ravel())
svc_1_train, svc_1_test = svc_1.score(Xtrain, ytrain.values.ravel()), svc_1.score(Xtest, ytest.values.ravel())

svc_grid = {'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'svc__gamma': ['scale'],
            'svc__C': [0.001, 0.01, 0.1, 1, 10, 100]}
svc_1_opt = GridSearchCV(svc_1, svc_grid, n_jobs=-2, cv=3)
svc_1_opt.fit(Xtrain, ytrain.values.ravel())
svc_1_opt_train, svc_1_opt_test, svc_1_opt_params = svc_1_opt.score(Xtrain, ytrain.values.ravel()), svc_1_opt.score(Xtest, ytest.values.ravel()), svc_1_opt.best_params_
print('Best Params:', svc_1_opt.best_params_)

In [56]:
scoring_table(scores)

Iteration0           Iteration1           Iteration2       \
                    NonScaled               Scaled                  PCA        
                        Train      Test      Train      Test      Train Test   
CatBoost default     0.997689  0.931608        NaN       NaN        NaN  NaN   
         optimized   1.000000  0.930684        NaN       NaN        NaN  NaN   
LightGBM default     1.000000  0.927911   1.000000  0.931608        NaN  NaN   
         optimized   1.000000  0.933457        NaN       NaN        NaN  NaN   
Logistic default          NaN       NaN   0.937616  0.922366        NaN  NaN   
         optimized        NaN       NaN   0.936922  0.922366        NaN  NaN   
KNN      default          NaN       NaN   0.944547  0.916821        NaN  NaN   
         optimized        NaN       NaN   1.000000  0.922366        NaN  NaN   
LDA      default          NaN       NaN   0.935074  0.920518        NaN  NaN   
         optimized        NaN       NaN   0.935536  0.921442        NaN  NaN   
GNB      default          NaN       NaN        NaN       NaN        NaN  NaN   
         optimized        NaN       NaN        NaN       NaN        NaN  NaN   
SVC      default          NaN       NaN        NaN       NaN        NaN  NaN   
         optimized        NaN       NaN        NaN       NaN        NaN  NaN   

                   Iteration3       
                     PCABoost       
                        Train Test  
CatBoost default          NaN  NaN  
         optimized        NaN  NaN  
LightGBM default          NaN  NaN  
         optimized        NaN  NaN  
Logistic default          NaN  NaN  
         optimized        NaN  NaN  
KNN      default          NaN  NaN  
         optimized        NaN  NaN  
LDA      default          NaN  NaN  
         optimized        NaN  NaN  
GNB      default          NaN  NaN  
         optimized        NaN  NaN  
SVC      default          NaN  NaN  
         optimized        NaN  NaN

## Iteration 2: Give the Computer a Chance

[Back to Top](#Machine-Learning-Processing)



In [ ]:
# IT2: PCA Only
from sklearn.decomposition import PCA 

In [ ]:
# IT2: CatBoost Classifier
cb_2 = Pipeline(steps=[('scaler', StandardScaler()),
                       ('PCA', PCA(n_components=0.95, random_state=430)),
                       ('catboost', CatBoostClassifier(random_state=430, verbose = False))])
cb_2.fit(Xtrain, ytrain)
cb_2_train, cb_2_test = cb_2.score(Xtrain, ytrain), cb_2.score(Xtest, ytest)

cb_2_opt = GridSearchCV(cb_2, cb_grid, n_jobs=-2, cv=3)
cb_2_opt.fit(Xtrain, ytrain)
cb_2_opt_train, cb_2_opt_test, cb_2_opt_params = cb_2_opt.score(Xtrain, ytrain), cb_2_opt.score(Xtest, ytest), cb_2_opt.best_params_

# print('CatBoost Train R2 Score:', cb_2_train)
# print('CatBoost Test R2 Score:', cb_2_test)
# print('CB Optimized Train Score:', cb_2_opt.score(Xtrain, ytrain))
# print('CB Optimized Test Score:', cb_2_opt.score(Xtest, ytest))
print('Best Params:', cb_2_opt.best_params_)

In [48]:
# IT2: Light Gradient Boosting Machine Classifier
lgbm_2 = Pipeline(steps=[('scaler', StandardScaler()),
                         ('PCA', PCA(n_components=0.95, random_state=430)),
                         ('lightgbm', LGBMClassifier(n_jobs=-2, random_state=430))])
lgbm_2.fit(Xtrain, ytrain.values.ravel())
lgbm_2_train, lgbm_2_test = lgbm_2.score(Xtrain, ytrain.values.ravel()), lgbm_2.score(Xtest, ytest.values.ravel())

lgbm_2_opt = GridSearchCV(lgbm_2, lgbm_grid, n_jobs=-2, cv=3)
lgbm_2_opt.fit(Xtrain, ytrain.values.ravel())
lgbm_2_opt_train, lgbm_2_opt_test, lgbm_2_opt_params = lgbm_2_opt.score(Xtrain, ytrain.values.ravel()), lgbm_2_opt.score(Xtest, ytest.values.ravel()), lgbm_2_opt.best_params_

# print('LightGBM Boost Train R2 Score:', lgbm_2_train)
# print('LightGBM Boost Test R2 Score:', lgbm_2_test)
# print('LightGBM Optimized Train Score:', lgbm_2_opt_train)
# print('LightGBM Optimized Test Score:', lgbm_2_opt_test)
print('Best Params:', lgbm_2_opt.best_params_)

KeyboardInterrupt: 

In [42]:
# IT2: Logistic Regression
lgr_2 = Pipeline(steps=[('scaler', StandardScaler()),
                        ('PCA', PCA(n_components=0.95, random_state=430)),
                        ('logistic', LogisticRegression(n_jobs=-2))])
lgr_2.fit(Xtrain, ytrain.values.ravel())
lgr_2_train, lgr_2_test = lgr_2.score(Xtrain, ytrain.values.ravel()), lgr_2.score(Xtest, ytest.values.ravel())

lgr_2_opt = GridSearchCV(lgr_2, lgr_grid, n_jobs=-2, cv=3)
lgr_2_opt.fit(Xtrain, ytrain.values.ravel())
lgr_2_opt_train, lgr_2_opt_test, lgr_2_opt_params = lgr_2_opt.score(Xtrain, ytrain.values.ravel()), lgr_2_opt.score(Xtest, ytest.values.ravel()), lgr_2_opt.best_params_
print('Best Params:', lgr_2_opt.best_params_)

C:\Users\gottl\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.93438152        nan 0.93484336
        nan 0.9343812         nan 0.93438088        nan 0.93414972
        nan 0.93368756        nan 0.93345639        nan 0.93299439
        nan 0.93299439        nan 0.93322555        nan 0.93322555
        nan 0.93322555        nan 0.93322555        n

Best Params: {'logistic__C': 0.24489795918367285, 'logistic__penalty': 'l2'}


In [43]:
# IT2: K-Nearest Neighbors
knn_2 = Pipeline(steps=[('scaler', StandardScaler()),
                        ('PCA', PCA(n_components=0.95, random_state=430)),
                        ('knn', KNeighborsClassifier(n_jobs=-2))])
knn_2.fit(Xtrain, ytrain.values.ravel())
knn_2_train, knn_2_test = knn_2.score(Xtrain, ytrain.values.ravel()), knn_2.score(Xtest, ytest.values.ravel())

knn_2_opt = GridSearchCV(knn_2, knn_grid, n_jobs=-2, cv=3)
knn_2_opt.fit(Xtrain, ytrain.values.ravel())
knn_2_opt_train, knn_2_opt_test, knn_2_opt_params = knn_2_opt.score(Xtrain, ytrain.values.ravel()), knn_2_opt.score(Xtest, ytest.values.ravel()), knn_2_opt.best_params_
print('Best Params:', knn_2_opt.best_params_)

Best Params: {'knn__metric': 'euclidean', 'knn__n_neighbors': 61, 'knn__weights': 'distance'}


In [45]:
# IT2: Linear Discriminant Analysis
lda_2 = Pipeline(steps=[('scaler', StandardScaler()),
                        ('PCA', PCA(n_components=0.95, random_state=430)),
                        ('lda', LinearDiscriminantAnalysis())])
lda_2.fit(Xtrain, ytrain.values.ravel())
lda_2_train, lda_2_test = lda_2.score(Xtrain, ytrain.values.ravel()), lda_2.score(Xtest, ytest.values.ravel())

lda_2_opt = GridSearchCV(lda_2, lda_grid, n_jobs=-2, cv=3)
lda_2_opt.fit(Xtrain, ytrain.values.ravel())
lda_2_opt_train, lda_2_opt_test, lda_2_opt_params = lda_2_opt.score(Xtrain, ytrain.values.ravel()), lda_2_opt.score(Xtest, ytest.values.ravel()), lda_2_opt.best_params_
print('Best Params:', lda_2_opt.best_params_)

Best Params: {'lda__solver': 'lsqr'}


In [ ]:
# IT2: Gaussian Naive Bayes
gnb_2 = Pipeline(steps=[('scaler', StandardScaler()),
                        ('PCA', PCA(n_components=0.95, random_state=430)),
                        ('gnb', GaussianNB())])
gnb_2.fit(Xtrain, ytrain.values.ravel())
gnb_2_train, gnb_2_test = gnb_2.score(Xtrain, ytrain.values.ravel()), gnb_2.score(Xtest, ytest.values.ravel())

gnb_2_opt = GridSearchCV(gnb_2, gnb_grid, n_jobs=-2, cv=3)
gnb_2_opt.fit(Xtrain, ytrain.values.ravel())
gnb_2_opt_train, gnb_2_opt_test, gnb_2_opt_params = gnb_2_opt.score(Xtrain, ytrain.values.ravel()), gnb_2_opt.score(Xtest, ytest.values.ravel()), gnb_2_opt.best_params_
print('Best Params:', gnb_2_opt.best_params_)

In [ ]:
# IT2: Support Vector Machine Classifier
svc_2 = Pipeline(steps=[('scaler', StandardScaler()),
                       ('PCA', PCA(n_components=0.95, random_state=430)),
                       ('svc', SVC())])
svc_2.fit(Xtrain, ytrain.values.ravel())
svc_2_train, svc_2_test = svc_2.score(Xtrain, ytrain.values.ravel()), svc_2.score(Xtest, ytest.values.ravel())

svc_2_opt = GridSearchCV(svc_2, svc_grid, n_jobs=-2, cv=3)
svc_2_opt.fit(Xtrain, ytrain.values.ravel())
svc_2_opt_train, svc_2_opt_test, svc_2_opt_params = svc_2_opt.score(Xtrain, ytrain.values.ravel()), svc_2_opt.score(Xtest, ytest.values.ravel()), svc_2_opt.best_params_
print('Best Params:', svc_2_opt.best_params_)

In [59]:
scoring_table(scores)

Iteration0           Iteration1           Iteration2       \
                    NonScaled               Scaled                  PCA        
                        Train      Test      Train      Test      Train Test   
CatBoost default     0.997689  0.931608        NaN       NaN        NaN  NaN   
         optimized   1.000000  0.930684        NaN       NaN        NaN  NaN   
LightGBM default     1.000000  0.927911   1.000000  0.931608        NaN  NaN   
         optimized   1.000000  0.933457        NaN       NaN        NaN  NaN   
Logistic default          NaN       NaN   0.937616  0.922366        NaN  NaN   
         optimized        NaN       NaN   0.936922  0.922366        NaN  NaN   
KNN      default          NaN       NaN   0.944547  0.916821        NaN  NaN   
         optimized        NaN       NaN   1.000000  0.922366        NaN  NaN   
LDA      default          NaN       NaN   0.935074  0.920518        NaN  NaN   
         optimized        NaN       NaN   0.935536  0.921442        NaN  NaN   
GNB      default          NaN       NaN        NaN       NaN        NaN  NaN   
         optimized        NaN       NaN        NaN       NaN        NaN  NaN   
SVC      default          NaN       NaN        NaN       NaN        NaN  NaN   
         optimized        NaN       NaN        NaN       NaN        NaN  NaN   

                   Iteration3       
                     PCABoost       
                        Train Test  
CatBoost default          NaN  NaN  
         optimized        NaN  NaN  
LightGBM default          NaN  NaN  
         optimized        NaN  NaN  
Logistic default          NaN  NaN  
         optimized        NaN  NaN  
KNN      default          NaN  NaN  
         optimized        NaN  NaN  
LDA      default          NaN  NaN  
         optimized        NaN  NaN  
GNB      default          NaN  NaN  
         optimized        NaN  NaN  
SVC      default          NaN  NaN  
         optimized        NaN  NaN

# PCA Boosting Technique
[Back to Top](#Machine-Learning-Processing)


## Iteration 3: PCABoosted Modeling
[Back to Top](#Machine-Learning-Processing)

Considering that the bespoke function would not play well with Pipeline, we have pre-processed the data here, first with scaling and then with the PCA Boost. Therefore, we can simply fit the raw models in this first, un-optimized modeling procedure.

## PCABoosted Report Card
[Back to Top](#Machine-Learning-Processing)

Rather than combine literally all of our information to this point, here I would like to summarize some of the best general performance and make an observation in the process: Pipelining PCA processing into a model generally seems to decrease overall testing performance across most models, which otherwise seemed to fare much better.

By contrast, PCA Boosting clearly features some potential and should be a great contrast to our preceding results. Therefore we will direct our focus to our results in Pipelined Scaling & PCA Boosting Procedures.

Compare

[Original Unoptimized](#Progress-Report-Card)

[Original Optimized](#Post-Optimization-Report-Card)

In [48]:
def PCAmerge(train, test, n_comp=0.95, r_state = 42):
    from sklearn.decomposition import PCA 
    pca = PCA(n_components = n_comp, random_state = r_state)
    train_pca = pd.DataFrame(pca.fit_transform(train), index=train.index)
    test_pca = pd.DataFrame(pca.transform(test), index=test.index)
    PCA_col_list = []
    for num in range(1, train_pca.shape[1]+1):
        PCA_col_list.append(('PCA' + str(num)))
    train_pca.columns = PCA_col_list
    train = pd.concat([train, train_pca], axis=1)
    test_pca.columns = PCA_col_list
    test = pd.concat([test, test_pca], axis=1)
    return train, test


In [49]:
# IT3: PCA Boosted Models
scaled = StandardScaler()
Xtrain_Scaled = pd.DataFrame(scaled.fit_transform(Xtrain), columns = Xtrain.columns, index = Xtrain.index)
Xtest_Scaled = pd.DataFrame(scaled.fit_transform(Xtest), columns = Xtest.columns, index = Xtest.index)
XtrainPCA, XtestPCA = PCAmerge(Xtrain_Scaled, Xtest_Scaled, n_comp=0.95)

In [ ]:
# IT3: CatBoost Classifier
cb_3 = CatBoostClassifier(random_state=430, verbose = False)
cb_3.fit(XtrainPCA, ytrain)
cb_3_train, cb_3_test = cb_3.score(XtrainPCA, ytrain), cb_3.score(XtestPCA, ytest)

cb_3_opt = GridSearchCV(cb_3, cb_grid, n_jobs=-2, cv=3)
cb_3_opt.fit(XtrainPCA, ytrain)
cb_3_opt_train, cb_3_opt_test, cb_3_opt_params = cb_3_opt.score(XtrainPCA, ytrain), cb_3_opt.score(XtestPCA, ytest), cb_3_opt.best_params_

# print('CatBoost Train R2 Score:', cb_3_train)
# print('CatBoost Test R2 Score:', cb_3_test)
# print('CB Optimized Train Score:', cb_3_opt.score(XtrainPCA, ytrain))
# print('CB Optimized Test Score:', cb_3_opt.score(XtestPCA, ytest))
print('Best Params:', cb_3_opt.best_params_)

In [ ]:
# IT3: Light Gradient Boosting Machine Classifier
lgbm_3 = LGBMClassifier(n_jobs=-2, random_state=430)
lgbm_3.fit(XtrainPCA, ytrain.values.ravel())
lgbm_3_train, lgbm_3_test = lgbm_3.score(XtrainPCA, ytrain.values.ravel()), lgbm_3.score(XtestPCA, ytest.values.ravel())

lgbm_3_opt = GridSearchCV(lgbm_3, lgbm_grid, n_jobs=-2, cv=3)
lgbm_3_opt.fit(XtrainPCA, ytrain.values.ravel())
lgbm_3_opt_train, lgbm_3_opt_test, lgbm_3_opt_params = lgbm_3_opt.score(XtrainPCA, ytrain.values.ravel()), lgbm_3_opt.score(XtestPCA, ytest.values.ravel()), lgbm_3_opt.best_params_

# print('LightGBM Boost Train R2 Score:', lgbm_3_train)
# print('LightGBM Boost Test R2 Score:', lgbm_3_test)
# print('LightGBM Optimized Train Score:', lgbm_3_opt_train)
# print('LightGBM Optimized Test Score:', lgbm_3_opt_test)
print('Best Params:', lgbm_3_opt.best_params_)

In [ ]:
# IT3: Logistic Regression
lgr_3 = LogisticRegression(n_jobs=-2)
lgr_3.fit(XtrainPCA, ytrain.values.ravel())
lgr_3_train, lgr_3_test = lgr_3.score(XtrainPCA, ytrain.values.ravel()), lgr_3.score(XtestPCA, ytest.values.ravel())

lgr_3_opt = GridSearchCV(lgr_3, lgr_grid, n_jobs=-2, cv=3)
lgr_3_opt.fit(XtrainPCA, ytrain.values.ravel())
lgr_3_opt_train, lgr_3_opt_test, lgr_3_opt_params = lgr_3_opt.score(XtrainPCA, ytrain.values.ravel()), lgr_3_opt.score(XtestPCA, ytest.values.ravel()), lgr_3_opt.best_params_
print('Best Params:', lgr_3_opt.best_params_)

In [ ]:
# IT3: K-Nearest Neighbors
knn_3 = KNeighborsClassifier(n_jobs=-2)
knn_3.fit(XtrainPCA, ytrain.values.ravel())
knn_3_train, knn_3_test = knn_3.score(XtrainPCA, ytrain.values.ravel()), knn_3.score(XtestPCA, ytest.values.ravel())

knn_3_opt = GridSearchCV(knn_3, knn_grid, n_jobs=-2, cv=3)
knn_3_opt.fit(XtrainPCA, ytrain.values.ravel())
knn_3_opt_train, knn_3_opt_test, knn_3_opt_params = knn_3_opt.score(XtrainPCA, ytrain.values.ravel()), knn_3_opt.score(XtestPCA, ytest.values.ravel()), knn_3_opt.best_params_
print('Best Params:', knn_3_opt.best_params_)

In [ ]:
# IT3: Linear Discriminant Analysis
lda_3 = LinearDiscriminantAnalysis()
lda_3.fit(XtrainPCA, ytrain.values.ravel())
lda_3_train, lda_3_test = lda_3.score(XtrainPCA, ytrain.values.ravel()), lda_3.score(XtestPCA, ytest.values.ravel())

lda_3_opt = GridSearchCV(lda_3, lda_grid, n_jobs=-2, cv=3)
lda_3_opt.fit(XtrainPCA, ytrain.values.ravel())
lda_3_opt_train, lda_3_opt_test, lda_3_opt_params = lda_3_opt.score(XtrainPCA, ytrain.values.ravel()), lda_3_opt.score(XtestPCA, ytest.values.ravel()), lda_3_opt.best_params_
print('Best Params:', lda_3_opt.best_params_)

In [ ]:
# IT3: Gaussian Naive Bayes
gnb_3 = GaussianNB()
gnb_3.fit(XtrainPCA, ytrain.values.ravel())
gnb_3_train, gnb_3_test = gnb_3.score(XtrainPCA, ytrain.values.ravel()), gnb_3.score(XtestPCA, ytest.values.ravel())

gnb_3_opt = GridSearchCV(gnb_3, gnb_grid, n_jobs=-2, cv=3)
gnb_3_opt.fit(XtrainPCA, ytrain.values.ravel())
gnb_3_opt_train, gnb_3_opt_test, gnb_3_opt_params = gnb_3_opt.score(XtrainPCA, ytrain.values.ravel()), gnb_3_opt.score(XtestPCA, ytest.values.ravel()), gnb_3_opt.best_params_
print('Best Params:', gnb_3_opt.best_params_)

In [ ]:
# IT3: Support Vector Machine Classifier
svc_3 = SVC()
svc_3.fit(XtrainPCA, ytrain.values.ravel())
svc_3_train, svc_3_test = svc_3.score(XtrainPCA, ytrain.values.ravel()), svc_3.score(XtestPCA, ytest.values.ravel())

svc_3_opt = GridSearchCV(svc_3, svc_grid, n_jobs=-2, cv=3)
svc_3_opt.fit(XtrainPCA, ytrain.values.ravel())
svc_3_opt_train, svc_3_opt_test, svc_3_opt_params = svc_3_opt.score(XtrainPCA, ytrain.values.ravel()), svc_3_opt.score(XtestPCA, ytest.values.ravel()), svc_3_opt.best_params_
print('Best Params:', svc_3_opt.best_params_)

In [ ]:
scoring_table(scores)

## Best Models

In [160]:
models = ['CatBoost', 'LightGBM', 'Logistic', 'KNN', 'LDA', 'GNB', 'SVC']
model_scores = scoring_table(scores)
test_scores = model_scores.loc[:, model_scores.columns.get_level_values(2) == 'Test'].unstack(1).T
best_models = pd.DataFrame([], columns = ['Iteration', 'Transformation', 'Test_Train', 'Optimized', 'Test Score'])
for model in models:
    best_models.loc[model] = test_scores[model].sort_values(ascending=False).reset_index().iloc[0].values
best_models = best_models.drop('Test_Train', axis=1)
best_models

,Iteration,Transformation,Optimized,Test Score
CatBoost,Iteration0,NonScaled,default,0.931608
LightGBM,Iteration0,NonScaled,optimized,0.933457
Logistic,Iteration1,Scaled,optimized,0.922366
KNN,Iteration1,Scaled,optimized,0.922366
LDA,Iteration1,Scaled,optimized,0.921442
GNB,Iteration0,NonScaled,default,NaN
SVC,Iteration0,NonScaled,default,NaN


In [206]:
modellistreduced = modellist.copy()

num_list = [105, 104]
i = 0
while i < 9:
    max_idx = len(num_list) - 1
    num_list.append(num_list[max_idx - 1] - 8)
    num_list.append(num_list[max_idx] - 8)
    i += 1

for num in num_list:
    modellistreduced.pop(num)
modellistreduced

['cb_0_train',
 'cb_0_test',
 'cb_1_train',
 'cb_1_test',
 'cb_2_train',
 'cb_2_test',
 'cb_3_train',
 'cb_3_test',
 'cb_0_opt_train',
 'cb_0_opt_test',
 'cb_1_opt_train',
 'cb_1_opt_test',
 'cb_2_opt_train',
 'cb_2_opt_test',
 'cb_3_opt_train',
 'cb_3_opt_test',
 'lgbm_0_train',
 'lgbm_0_test',
 'lgbm_1_train',
 'lgbm_1_test',
 'lgbm_2_train',
 'lgbm_2_test',
 'lgbm_3_train',
 'lgbm_3_test',
 'lgbm_0_opt_train',
 'lgbm_0_opt_test',
 'lgbm_1_opt_train',
 'lgbm_1_opt_test',
 'lgbm_2_opt_train',
 'lgbm_2_opt_test',
 'lgbm_3_opt_train',
 'lgbm_3_opt_test',
 'lgr_1_train',
 'lgr_1_test',
 'lgr_2_train',
 'lgr_2_test',
 'lgr_3_train',
 'lgr_3_test',
 'lgr_1_opt_train',
 'lgr_1_opt_test',
 'lgr_2_opt_train',
 'lgr_2_opt_test',
 'lgr_3_opt_train',
 'lgr_3_opt_test',
 'knn_1_train',
 'knn_1_test',
 'knn_2_train',
 'knn_2_test',
 'knn_3_train',
 'knn_3_test',
 'knn_1_opt_train',
 'knn_1_opt_test',
 'knn_2_opt_train',
 'knn_2_opt_test',
 'knn_3_opt_train',
 'knn_3_opt_test',
 'lda_1_train',
 'ld

In [215]:
modelinputlist = modellistreduced.copy()
for idx in range(len(modelinputlist)):
    if idx % 2 == 0:
        modelinputlist[idx] = modelinputlist[idx][:-6]
    else:
        modelinputlist[idx] = modelinputlist[idx][:-5]
modelinputlist

['cb_0',
 'cb_0',
 'cb_1',
 'cb_1',
 'cb_2',
 'cb_2',
 'cb_3',
 'cb_3',
 'cb_0_opt',
 'cb_0_opt',
 'cb_1_opt',
 'cb_1_opt',
 'cb_2_opt',
 'cb_2_opt',
 'cb_3_opt',
 'cb_3_opt',
 'lgbm_0',
 'lgbm_0',
 'lgbm_1',
 'lgbm_1',
 'lgbm_2',
 'lgbm_2',
 'lgbm_3',
 'lgbm_3',
 'lgbm_0_opt',
 'lgbm_0_opt',
 'lgbm_1_opt',
 'lgbm_1_opt',
 'lgbm_2_opt',
 'lgbm_2_opt',
 'lgbm_3_opt',
 'lgbm_3_opt',
 'lgr_1',
 'lgr_1',
 'lgr_2',
 'lgr_2',
 'lgr_3',
 'lgr_3',
 'lgr_1_opt',
 'lgr_1_opt',
 'lgr_2_opt',
 'lgr_2_opt',
 'lgr_3_opt',
 'lgr_3_opt',
 'knn_1',
 'knn_1',
 'knn_2',
 'knn_2',
 'knn_3',
 'knn_3',
 'knn_1_opt',
 'knn_1_opt',
 'knn_2_opt',
 'knn_2_opt',
 'knn_3_opt',
 'knn_3_opt',
 'lda_1',
 'lda_1',
 'lda_2',
 'lda_2',
 'lda_3',
 'lda_3',
 'lda_1_opt',
 'lda_1_opt',
 'lda_2_opt',
 'lda_2_opt',
 'lda_3_opt',
 'lda_3_opt',
 'gnb_1',
 'gnb_1',
 'gnb_2',
 'gnb_2',
 'gnb_3',
 'gnb_3',
 'gnb_1_opt',
 'gnb_1_opt',
 'gnb_2_opt',
 'gnb_2_opt',
 'gnb_3_opt',
 'gnb_3_opt',
 'svc_1',
 'svc_1',
 'svc_2',
 'svc_2',


In [169]:
dictlist = [ele for ele in ['cb', 'lgbm', 'lgr', 'knn', 'lda', 'gnb', 'svc'] for i in range(4)]
iterdictlist = ['_0', '_1', '_2', '_3'] * 7
for idx in range(len(dictlist)):
    dictlist[idx] = dictlist[idx] + iterdictlist[idx] + '_opt_params'
for num in [24, 20, 16, 12, 8]:
    dictlist.pop(num)

['cb_0_opt_params',
 'cb_1_opt_params',
 'cb_2_opt_params',
 'cb_3_opt_params',
 'lgbm_0_opt_params',
 'lgbm_1_opt_params',
 'lgbm_2_opt_params',
 'lgbm_3_opt_params',
 'lgr_1_opt_params',
 'lgr_2_opt_params',
 'lgr_3_opt_params',
 'knn_1_opt_params',
 'knn_2_opt_params',
 'knn_3_opt_params',
 'lda_1_opt_params',
 'lda_2_opt_params',
 'lda_3_opt_params',
 'gnb_1_opt_params',
 'gnb_2_opt_params',
 'gnb_3_opt_params',
 'svc_1_opt_params',
 'svc_2_opt_params',
 'svc_3_opt_params']

In [216]:
model_dict = {}
all_list = modelinputlist + modellistreduced + dictlist
for ele in all_list:
    model_dict[ele] = locals()[ele]

KeyError: 'cb_1'

In [ ]:
pickle.dump(model_scores, open('model_scores.pickle', 'wb'))
pickle.dump(best_models, open('best_models.pickle', 'wb'))
pickle.dump(model_dict, open('model_dict.pickle', 'wb'))

## Running the Best Models for Full Training Data

In [5]:
##### If starting from here, uncomment these #####
# import numpy as np
# import pandas as pd
# import pickle

# from sklearn.pipeline import Pipeline
# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC
# from sklearn.decomposition import PCA 


# def PCAmerge(train, test, n_comp=0.95, r_state = 42):
#     from sklearn.decomposition import PCA 
#     pca = PCA(n_components = n_comp, random_state = r_state)
#     train_pca = pd.DataFrame(pca.fit_transform(train), index=train.index)
#     test_pca = pd.DataFrame(pca.transform(test), index=test.index)
#     PCA_col_list = []
#     for num in range(1, train_pca.shape[1]+1):
#         PCA_col_list.append(('PCA' + str(num)))
#     train_pca.columns = PCA_col_list
#     train = pd.concat([train, train_pca], axis=1)
#     test_pca.columns = PCA_col_list
#     test = pd.concat([test, test_pca], axis=1)
#     return train, test


# scaled = StandardScaler()
# Xtrain_Scaled = pd.DataFrame(scaled.fit_transform(Xtrain), columns = Xtrain.columns, index = Xtrain.index)
# Xtest_Scaled = pd.DataFrame(scaled.fit_transform(Xtest), columns = Xtest.columns, index = Xtest.index)
# XtrainPCA, XtestPCA = PCAmerge(Xtrain_Scaled, Xtest_Scaled, n_comp=0.95)

# pickle.load(open('model_scores.pickle', 'rb'))
# pickle.load(open('best_models.pickle', 'rb'))
# pickle.load(open('model_dict.pickle', 'rb'))

In [3]:
Xtrain_all = pickle.load(open('Xtrain.pickle', 'rb'))
ytrain_all = pickle.load(open('ytrain.pickle', 'rb'))
Xtest_all = pickle.load(open('Xtest.pickle', 'rb'))
ytest_all = pickle.load(open('ytest.pickle', 'rb'))

# Results & Discussion
[Back to Top](#Machine-Learning-Processing)

To recap, much of our exploration here has been to probe the usefulness of unsupervised learning in a pipelined format versus its application as a boosting procedure. Simultaneously, we have employed a diverse number of model types, including some newer devices in the CatBoostClassifier and the LightGBMClassifier.

From much of our results, we have revealed that both uses of PCA have their respective uses, the latter, the PCA Boosting Procedure, seems to have the greatest effect of narrowing overfit results. However, the greatest results of accuracy come to us through the raw, LightGBM results.

With that said, we will now turn our attention to the matter of insight from our findings. To whit, we will break this down into three parts:
   - Feature Importance
   - Prediction Refinement
   - Business Relevance & Benefits